<h1>SI 670 Kaggle Competition 4</h1>

Team GSIs

In [1]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import Model, Sequential, layers
from tensorflow.keras.losses import BinaryCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import Input 

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

<h2>Load datasets</h2>

In [3]:
len(list(os.listdir("../data/images_train/bathroom"))), len(list(os.listdir("../data/images_train/kitchen")))

(301, 315)

In [4]:
data = tf.keras.utils.image_dataset_from_directory("../data/images_train", 
                                                   image_size = (256, 256), 
                                                   class_names = ['bathroom', 'kitchen'], 
                                                   shuffle = True,
                                                   batch_size = 32)

Found 616 files belonging to 2 classes.


2023-11-18 21:18:02.421850: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2023-11-18 21:18:02.421879: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 24.00 GB
2023-11-18 21:18:02.421888: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 8.00 GB
2023-11-18 21:18:02.421927: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-18 21:18:02.421999: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
X, y = zip(*data)

X = np.concatenate(X)
y = np.concatenate(y)

In [6]:
len(X), len(y)

(616, 616)

<h2>Preprocess Images</h2>

In [7]:
#flip labels so bathroom is 1
y = np.where(y == 1, 0, 1)

In [8]:
X = np.apply_along_axis(lambda s: s / 255.0, axis = 1, arr = X)

<h2>Split</h2>

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state = 42, test_size = 0.2)

<h2>Train Model</h2>

In [10]:
model = Sequential([
    # input
    tf.keras.layers.Conv2D(64, (3, 3), kernel_regularizer = l2(0.01), input_shape = (256, 256, 3)), 
    tf.keras.layers.BatchNormalization(axis = -1, epsilon = 2e-5),  
    tf.keras.layers.Conv2D(128, (3, 3), kernel_regularizer = l2(0.01), activation = 'relu'), 
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.MaxPooling2D((3, 3)),
    tf.keras.layers.BatchNormalization(axis = -1, epsilon = 2e-5, input_shape = (256, 256, 3)), 
    tf.keras.layers.Conv2D(128, (3, 3), kernel_regularizer = l2(0.01), activation = 'relu'), 
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.BatchNormalization(axis = -1, epsilon = 2e-5, input_shape = (256, 256, 3)), 
    tf.keras.layers.Conv2D(128, (3, 3), kernel_regularizer = l2(0.01), activation = 'relu'), 
    tf.keras.layers.Conv2D(256, (3, 3), kernel_regularizer = l2(0.01), activation = 'relu'), 
    # tf.keras.layers.Conv2D(128, (3, 3), kernel_regularizer = l2(0.01), activation = 'relu'), 
    # tf.keras.layers.MaxPooling2D((2, 2)),
    # tf.keras.layers.Conv2D(128, (3, 3), kernel_regularizer = l2(0.01), activation = 'relu'), 
    # tf.keras.layers.Dropout(0.1),
    # tf.keras.layers.Conv2D(64, (3, 3), kernel_regularizer = l2(0.001), activation = 'relu'),
    # tf.keras.layers.MaxPooling2D((2, 2)),
    # tf.keras.layers.Conv2D(32, (3, 3), kernel_regularizer = l2(0.01), activation = 'relu'), 
    tf.keras.layers.Conv2D(32, (3, 3), kernel_regularizer = l2(0.01), activation = 'relu'), 
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(64), 
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [ ]:
loss = BinaryCrossentropy()
model.compile(optimizer = 'adam', loss = loss, metrics = ['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor = 'val_loss', min_delta = 0.005, patience = 3)

model_history = model.fit(X_train, y_train, epochs = 50, batch_size = 8, 
                          callbacks = [early_stopping], 
                          validation_data = (X_val, y_val))

In [ ]:
model.save("../saved_weights/haley_submit_6")

<h2>Test Model</h2>

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
model.evaluate(X_val, y_val)

In [ ]:
y_pred_prob = model.predict(X_test)

In [ ]:
y_pred_class = np.where(y_pred_prob > 0.5, 1, 0).flatten()

In [ ]:
matrix = confusion_matrix(y_pred_class, y_test)

In [ ]:
normalized_confusion_matrix = matrix / matrix.astype(float).sum(axis = 1)[:, np.newaxis]

In [ ]:
f = sns.heatmap(normalized_confusion_matrix, annot = True)
f.set(title = 'Normalized Confusion Matrix', xlabel = 'True', ylabel = 'Predicted')
plt.savefig("../plots/haley_heatmap_submission6.png")

<h2>Make Prediction</h2>

In [ ]:
pred_data = tf.keras.utils.image_dataset_from_directory("../data/images_test", image_size = (256, 256), labels = None)
pred_data = pred_data.unbatch()
pred_images = [img / 255.0 for img in pred_data]

In [ ]:
imgs = []
ids = []

for f in os.listdir("../data/images_test"):
    img = tf.keras.utils.load_img(os.path.join("../data/images_test", f), target_size = (256, 256, 3))
    img = tf.keras.utils.img_to_array(img) / 255.0
    id = f.split(".")[0]
    ids.append(id)
    imgs.append(img)

In [ ]:
predictions = model.predict(np.array(np.array(imgs)))

<h2>Format predictions for submission</h2>

In [ ]:
class_predictions = np.where(predictions > 0.5, 1, 0).flatten()

In [ ]:
df = pd.DataFrame({'id': ids, 'class': class_predictions})

In [ ]:
df.to_csv("../submissions/submit_6.csv", index = None)

In [ ]:
x = np.array([[3, 3, 3], [2, 2, 3], [1, 1, 1]])

In [ ]:
x